In [2]:
import pandas as pd
import string
import Levenshtein as levenshtein
import json

In [13]:
# Movie Info
# H:/DataIntegation/metacritic/movieDatasetClean.csv

df_metacritic_movie = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/metacritic/movieDatasetClean.csv")
df_metacritic_movie.head()

,title,age_rating,rating,rank,genre,director,year,producer,actor,runtime,description,img,url
0,Citizen Kane,15,8.4,1,Drama,Orson Welles,1941,RKO Radio Pictures,Joseph Cotten,119,"Following the death of a publishing tycoon, ne...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/citizen-kane
1,The Godfather,18,9.2,2,Drama,Francis Ford Coppola,1972,Paramount Pictures,Al Pacino,175,Francis Ford Coppola's epic features Marlon Br...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/the-godfather
2,Rear Window,0,8.8,3,Mystery,Alfred Hitchcock,1954,Paramount Pictures,Frank Cady,112,A wheelchair-bound photographer spies on his n...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/rear-window
3,Casablanca,15,8.9,4,Drama,Michael Curtiz,1943,Warner Bros.,Humphrey Bogart,102,"A Casablanca, Morocco casino owner in 1941 she...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/casablanca
4,Boyhood,18,7.5,5,Drama,Richard Linklater,2014,IFC Films,Bonnie Cross,165,"Filmed over 12 years with the same cast, Richa...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/boyhood


In [4]:
df_metacritic_review = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/metacritic/reviewDatasetClean.csv",  lineterminator='\n')
df_metacritic_review.head()

,title,user,type,grade,review,spoilers,language
0,Citizen Kane,larsenonfilm,reviewer,100,...,0,en
1,Citizen Kane,chicago-sun-times,reviewer,100,...,0,en
2,Citizen Kane,new-york-daily-news,reviewer,100,...,0,en
3,Citizen Kane,reelviews,reviewer,100,...,0,en
4,Citizen Kane,empire,reviewer,100,...,0,en


In [14]:
df_metacritic_movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14213 entries, 0 to 14212
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        14213 non-null  object
 1   age_rating   14213 non-null  int64 
 2   rating       14213 non-null  object
 3   rank         14213 non-null  int64 
 4   genre        14213 non-null  object
 5   director     14213 non-null  object
 6   year         14213 non-null  int64 
 7   producer     13941 non-null  object
 8   actor        14213 non-null  object
 9   runtime      14213 non-null  int64 
 10  description  14210 non-null  object
 11  img          14213 non-null  object
 12  url          14213 non-null  object
dtypes: int64(4), object(9)
memory usage: 1.4+ MB


Kết quả Schema Matching: (Những trường bị đánh * trong Source Schema là bị matching sai)

- Bảng Movie:

| Source Schema | Data warehouse Schema |
|---------------|-----------------------|
| url*          | movie_title           |
| rating*       | content_rating        |
| genre         | genres                |
| director      | directors             |
| producer      | production_company    |
| actor         | actors                |
| runtime       | runtime               |
| description   | movie_info            |
| img*          | release_date          |

- Bảng Rating:

| Source Schema | Data warehouse Schema |
|---------------|-----------------------|
| rating        | avg_rating            |
| year*         | rating_count          |
| url           | link                  |

- Bảng Review

| Source Schema | Data warehouse Schema |
|---------------|-----------------------|
| title         | movie_id              |
| review        | review_content        |
| spoiler*      | review_score          |
| language*     | review_date           |

Nhận xét và hướng xử lý của các trường:

- Bảng Movie:
    + Trường content_rating trong data warehouse bị matching sai, trường đúng phải là age_rating, lý do sai là do age_rating  trong dữ liệu Metacritic đang để dưới dạng số => chuyển về định dạng chung giống bảng tham khảo content_rating
    + Trường movie_title trong data warehouse bị matching sai, trường đúng phải là title (Lý do sai chưa rõ) => khi xem xét lại không ảnh hưởng nhiều => không cần điều chỉnh
    + Trường release_date trong data warehouse bị matching sai, trường đúng phải là year, lý do sai là do year trong dữ liệu Metacritic đang để dưới dạng năm, trong khi phần lớn dữ liệu có trong training data để dưới dạng Year/Month/Day. Tuy nhiên, ta thấy dữ liệu release_year trong metacritic được viết đúng dạng, nhưng lại không đúng kiểu dữ liệu (string vs float) => chuyển đổi kiểu dữ liệu cho trường year trong metacritic
    + Trường genre, director, producer, actor được viết đúng dạng, tuy nhiên giá trị các trường trong Metacritic có thể khác so với trong Data Warehouse => cần phải biến đổi lại
- Bảng Rating:
    + Kiểm tra lại dữ liệu movie Metacritic, ta nhận thấy không có trường nào thể hiện số lượt rating, ta sẽ phải tạo trường đó thông qua dữ liệu review của Metacritic
- Bảng Review:
    + Trường review_score trong data warehouse bị matching sai, trường đúng phải là grade (Lý do sai chưa rõ)
    + Trường movie_id trong data warehouse hiện tại đang được match vào trường title trong metacritic => matching đúng, tuy nhiên ta sẽ phải thêm trường movie_id cho 2 bảng movie và review
    + Thông tin về review date không có trong nguồn Metacritic 

## Trường year

In [16]:
def change_type(year):
    if type(year)!=float:
        return float(year)
df_metacritic_movie["year"]=[change_type(i) for i in df_metacritic_movie["year"]]

## Trường age_rating

In [100]:
df_metacritic_movie['age_rating'].unique()

array([15, 18,  0, -5, 13, 17, 14,  7])

In [102]:
warehouse_content_rating_list = list(pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/content_rating.csv")['content_rating'])
warehouse_content_rating_list

['PG', 'R', 'NR', 'G', 'PG-13', 'NC17']

Ta thực hiện Mapping Content Rating như dưới:
- G: 0
- PG-13: 7 
- R: 13, 14, 15 
- NC-17: 17, 18 
- NR: -5 

In [103]:
replace_age_rating_dict = { 0 : 'G', 7 : 'PG-13', 13 : 'R', 14: 'R', 15: 'R', 17: 'NC-17', 18: 'NC-17', -5: 'NR'}
def preprocess_age_rating(age_rating):
    age_rating = replace_age_rating_dict[age_rating]
    return age_rating
df_metacritic_movie['age_rating'] = [preprocess_age_rating(i) for i in df_metacritic_movie['age_rating']]
df_metacritic_movie['age_rating'].unique()

array(['R', 'NC-17', 'G', 'NR', 'PG-13'], dtype=object)

In [104]:
df_metacritic_movie.head()

,title,age_rating,rating,rank,genre,director,year,producer,actor,runtime,description,img,url
0,Citizen Kane,R,8.4,1,Drama,Orson Welles,1941,RKO Radio Pictures,Joseph Cotten,119,"Following the death of a publishing tycoon, ne...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/citizen-kane
1,The Godfather,NC-17,9.2,2,Drama,Francis Ford Coppola,1972,Paramount Pictures,Al Pacino,175,Francis Ford Coppola's epic features Marlon Br...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/the-godfather
2,Rear Window,G,8.8,3,Mystery,Alfred Hitchcock,1954,Paramount Pictures,Frank Cady,112,A wheelchair-bound photographer spies on his n...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/rear-window
3,Casablanca,R,8.9,4,Drama,Michael Curtiz,1943,Warner Bros.,Humphrey Bogart,102,"A Casablanca, Morocco casino owner in 1941 she...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/casablanca
4,Boyhood,NC-17,7.5,5,Drama,Richard Linklater,2014,IFC Films,Bonnie Cross,165,"Filmed over 12 years with the same cast, Richa...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/boyhood


## Trường genre

In [105]:
df_metacritic_movie["genre"].unique()

array(['Drama', 'Mystery', 'Comedy', 'Fantasy', 'Adventure', 'Action',
       'Biography', 'Documentary', 'Sci-Fi', 'History', 'Thriller',
       'Western', 'Music', 'Crime', 'War', 'Horror', 'Romance',
       'Animation', 'Family', 'Sport', 'Musical', 'untagged'],
      dtype=object)

Ta thấy trường genre trong Metacritic chỉ có 1 giá trị => không cần biến đổi list

Ta nhận thấy trong genre có trường untagged, ta sẽ sửa thành None

In [6]:
warehouse_genres_list = list(pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/genres.csv")["genres"])
warehouse_genres_list

['Faith & Spirituality',
 'Cult Movies',
 'Western',
 'Thrill',
 'Horror',
 'Special Interest',
 'Science Fiction & Fantasy',
 'Gay & Lesbian',
 'Drama',
 'Documentary',
 'War',
 'Romance',
 'Crime',
 'Film-Noir',
 'Comedy',
 'Animation',
 'Musical & Performing Arts',
 'Action & Adventure',
 'Mystery & Suspense',
 'Sports & Fitness',
 'Kids & Family',
 'IMAX',
 'Anime & Manga',
 'Television',
 'Art House & International',
 'Classics']

Từ 2 danh sách thể loại, ta có thể mapping như dưới:
|Data warehouse Genres|Metacritic Genres |
| :- | :- |
|‘Classic’||
|‘Documentary’|‘Documentary’|
|‘Western’|‘Western’|
|‘Horror’|‘Horror’|
|‘Science Fiction & Fantasy’ *|‘Sci-Fi’, ‘Fantasy’|
|‘Gay & Lesbian’||
|‘Drama’|‘Drama’|
|‘Comedy’|‘Comedy’|
|‘Cult Movies’||
|‘Romance’|‘Romance’|
|‘Television’||
|‘Sport & Fitness’ *|‘Sport’|
|‘Art House & International’||
|‘Special Interest’||
|‘Animation’|‘Animation’|
|‘Musical & Performing Arts’ *|‘Music’, ‘Musical’|
|‘Faith & Spirituality’||
|‘Mystery & Suspense’ *|‘Mystery’|
|‘Anime & Manga’||
|‘Action & Adventure’ *|‘Action’, ‘Adventure’|
|‘Kids & Family’ *|‘Family’|
|‘War’|‘War’|
|‘Crime’|‘Crime’|
|‘Film-Noir’||
|‘IMAX’||
|‘Thrill’ *|‘Thriller’|

Những giá trị được đánh * là những giá trị cần phải chuyển đổi

Những giá trị như Biography, History ta sẽ thêm vào Data Warehouse

In [106]:
replace_genres_dict = {'Mystery' : 'Mystery & Suspense', 'Thriller': 'Thrill',
                        'Sci-Fi': 'Science Fiction & Fantasy', 'Fantasy': 'Science Fiction & Fantasy',
                       'Adventure': 'Action & Adventure','Action': 'Action & Adventure',
                       'Musical': 'Musical & Performing Arts', 'Music': 'Musical & Performing Arts',
                       'Sport': 'Sport & Fitness',
                        'Family': 'Kids & Family'}
def preprocess_genres(genre):
    if genre == 'untagged':
        return None
    if genre in replace_genres_dict:
        genre = replace_genres_dict[genre]
    return genre
df_metacritic_movie["genre"] = [preprocess_genres(i) for i in df_metacritic_movie["genre"]]

In [10]:
# Thêm các giá trị mới vào trong genres warehouse
warehouse_genres_list+=["Biography", "History"]
warehouse_genres_list = list(set(warehouse_genres_list))
pd.DataFrame(warehouse_genres_list, columns=["genres"]).to_csv(
    "/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/genres.csv",
    index=False 
)

# Trường actor và director

In [119]:
warehouse_directors_actors_list = list(pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/directors_actors.csv")["name"])
print("Số giá trị của trường directors và actors riêng biệt có trong warehouse: "+ str(len(warehouse_directors_actors_list)))

Số giá trị của trường directors và actors riêng biệt có trong warehouse: 210926


In [87]:
print("Số giá trị director riêng biệt trong metacritic: "+str(len(df_metacritic_movie["director"].unique())))
print("Số giá trị director bị null trong metacritic: "+str(sum(df_metacritic_movie["director"].isna())))
print("Số giá trị actor riêng biệt trong metacritic: "+str(len(df_metacritic_movie["actor"].unique())))
print("Số giá trị actor bị null trong metacritic: "+str(sum(df_metacritic_movie["actor"].isna())))

Số giá trị director riêng biệt trong metacritic: 6666
Số giá trị director bị null trong metacritic: 0
Số giá trị actor riêng biệt trong metacritic: 5733
Số giá trị actor bị null trong metacritic: 0


Ta thấy trường actor và director trong metacritic cũng chỉ chứa 1 giá trị => không cần biến đổi list

Do có rất nhiều giá trị trong trường director và actor của Metacritic và trường directors và actors của Warehouse => Sử dụng String Matching

Các bước thực hiện:
- Ứng với mỗi giá trị của trường production_company trong Warehouse, ta sẽ lọc ra các ứng cử viên các giá trị trong trường producer của Metacritic (Bằng chữ cái đầu tiên)
- Sử dụng String Matching để so khớp giá trị đang xét của trường production_company với các giá trị ứng cử viên của trường producer của Metacritic 

Cụ thể hơn về chiến lược lọc, nếu từ đầu tiên trong string là chữ cái hoặc chữ số thì ta sẽ xét từ đầu tiên (phải lowercase), nếu là dấu câu thì ta sẽ xét toàn bộ

## Trường director

In [57]:
def get_candidate(source, first_character):
    if first_character in string.punctuation:
        return source
    else:
        candidate = []
        for i in source:
            if type(i)==str:
                if i[0].lower() == first_character:
                    candidate.append(i)
                elif i[0] in string.punctuation:
                    candidate.append(i)
        return candidate

In [88]:
director_mapping = {}
number_of_case = []
for idx, i in enumerate(warehouse_directors_actors_list):
    if idx % 30000 == 0:
        print(idx)
    candidate = get_candidate(df_metacritic_movie["director"].unique(), i[0].lower())
    number_of_case.append(len(candidate))
    match=[]
    for j in candidate:
        if levenshtein.ratio(i, j)>=0.9:
            match.append(j)
    director_mapping[i]=match

0
30000
60000
90000
120000
150000
180000
210000


In [92]:
count=0
for key, value in director_mapping.items():
    if len(value)!=0:
        count+=1
        print("directors (Data warehouse): "+key)
        print("director (Metacritic): "+str(value))
        print()
        if count==10:
            break

directors (Data warehouse): Kevin Scott
director (Metacritic): ['Ken Scott']

directors (Data warehouse): Naji Abu Nowar
director (Metacritic): ['Naji Abu Nowar']

directors (Data warehouse): Tony McNamara
director (Metacritic): ['Tony McNamara']

directors (Data warehouse): Gaël Morel
director (Metacritic): ['Gaël Morel']

directors (Data warehouse): Lexi Alexander
director (Metacritic): ['Lexi Alexander']

directors (Data warehouse): David McNally
director (Metacritic): ['David McNally']

directors (Data warehouse): Niels Tavernier
director (Metacritic): ['Nils Tavernier']

directors (Data warehouse): Thomas Schlamme
director (Metacritic): ['Thomas Schlamme']

directors (Data warehouse): Julia Hart
director (Metacritic): ['Julia Hart']

directors (Data warehouse): Tony Roberts
director (Metacritic): ['Ty Roberts']



In [94]:
director_mapping_dict = {}
for key, value in director_mapping.items():
    if len(value)!=0:
        for i in value:
            director_mapping_dict[i]=key
# Lưu lại các giá trị director chưa có trong warehouse 
remain_director = [item for item in list(df_metacritic_movie["director"].unique()) if item not in list(director_mapping_dict.keys())]

In [107]:
# Chuyển đổi giá trị trường director trong metacritic về giá trị trường directors trong data warehouse
def preprocess_name(name, replace_name_dict):
    if name in replace_name_dict:
        name = replace_name_dict[name]
    return name
df_metacritic_movie["director"] = [preprocess_name(i, director_mapping_dict) for i in df_metacritic_movie["director"]]

## Trường actor

In [96]:
actor_mapping = {}
number_of_case = []
for idx, i in enumerate(warehouse_directors_actors_list):
    if idx % 30000 == 0:
        print(idx)
    candidate = get_candidate(df_metacritic_movie["actor"].unique(), i[0].lower())
    number_of_case.append(len(candidate))
    match=[]
    for j in candidate:
        if levenshtein.ratio(i, j)>=0.9:
            match.append(j)
    actor_mapping[i]=match

0
30000
60000
90000
120000
150000
180000
210000


In [98]:
count=0
for key, value in actor_mapping.items():
    if len(value)!=0:
        count+=1
        print("actors (Data warehouse): "+key)
        print("actor (Metacritic): "+str(value))
        print()
        if count==10:
            break

actors (Data warehouse): Jason Statham
actor (Metacritic): ['Jason Statham']

actors (Data warehouse): Charley Grapewin
actor (Metacritic): ['Charley Grapewin']

actors (Data warehouse): Alex Knight
actor (Metacritic): ['Alex Knight']

actors (Data warehouse): Angeliki Papoulia
actor (Metacritic): ['Aggeliki Papoulia']

actors (Data warehouse): Charlie Day
actor (Metacritic): ['Charlie Day']

actors (Data warehouse): John Rutter
actor (Metacritic): ['John Ritter']

actors (Data warehouse): Allan Arbus
actor (Metacritic): ['Allan Arbus']

actors (Data warehouse): Dave Geolz
actor (Metacritic): ['Dave Goelz']

actors (Data warehouse): Casper Van Dien
actor (Metacritic): ['Casper Van Dien']

actors (Data warehouse): André Marcon
actor (Metacritic): ['André Marcon']



In [108]:
actor_mapping_dict = {}
for key, value in actor_mapping.items():
    if len(value)!=0:
        for i in value:
            actor_mapping_dict[i]=key
# Lưu lại các giá trị actor chưa có trong warehouse 
remain_actor = [item for item in list(df_metacritic_movie["actor"].unique()) if item not in list(actor_mapping_dict.keys())]
# Chuyển đổi giá trị trường director trong metacritic về giá trị trường directors trong data warehouse
df_metacritic_movie["actor"] = [preprocess_name(i, actor_mapping_dict) for i in df_metacritic_movie["actor"]]

## Lưu các giá trị director và actor mới vào data warehouse

In [ ]:
# Lưu director mapping và actor mapping
with open('/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/metacritic/mapping/director_mapping.json', "w") as write_file:
    json.dump(director_mapping, write_file, indent=4)
with open('/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/metacritic/mapping/actor_mapping.json', "w") as write_file:
    json.dump(actor_mapping, write_file, indent=4)

In [ ]:
warehouse_directors_actors_list+=remain_actor
warehouse_directors_actors_list+=remain_director
warehouse_directors_actors_list=list(set(warehouse_directors_actors_list))
pd.DataFrame(warehouse_directors_actors_list, columns=["name"]).to_csv(
    "/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/directors_actors.csv",
    index=False 
)

# Tạo trường rating_count

Ta tính ratecount thông qua bảng df_metacritic_review 

In [109]:
df_metacritic_review = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/metacritic/reviewDatasetClean.csv", 
                                   lineterminator='\n')
df_metacritic_review.head()

,title,user,type,grade,review,spoilers,language
0,Citizen Kane,larsenonfilm,reviewer,100,...,0,en
1,Citizen Kane,chicago-sun-times,reviewer,100,...,0,en
2,Citizen Kane,new-york-daily-news,reviewer,100,...,0,en
3,Citizen Kane,reelviews,reviewer,100,...,0,en
4,Citizen Kane,empire,reviewer,100,...,0,en


In [111]:
df_metacritic_statistic = pd.DataFrame()
df_metacritic_statistic["title"] = df_metacritic_review.groupby("title")["grade"].count().keys()
df_metacritic_statistic["ratecount"] = df_metacritic_review.groupby("title")["grade"].count().values
df_metacritic_statistic.head()

,title,ratecount
0,!Women Art Revolution,11
1,#Horror,14
2,$9.99,17
3,$pent,8
4,'71,61


In [113]:
df_metacritic_final = pd.concat([df_metacritic_movie, df_metacritic_statistic], axis=1)
df_metacritic_final

,title,age_rating,rating,rank,genre,director,year,producer,actor,runtime,description,img,url,title,ratecount
0,Citizen Kane,R,8.4,1,Drama,Orson Welles,1941,RKO Radio Pictures,Joseph Cotten,119,"Following the death of a publishing tycoon, ne...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/citizen-kane,!Women Art Revolution,11.0
1,The Godfather,NC-17,9.2,2,Drama,Francis Ford Coppola,1972,Paramount Pictures,Al Pacino,175,Francis Ford Coppola's epic features Marlon Br...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/the-godfather,#Horror,14.0
2,Rear Window,G,8.8,3,Mystery & Suspense,Alfred Hitchcock,1954,Paramount Pictures,Frank Cady,112,A wheelchair-bound photographer spies on his n...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/rear-window,$9.99,17.0
3,Casablanca,R,8.9,4,Drama,Michael Curtiz,1943,Warner Bros.,Humphrey Bogart,102,"A Casablanca, Morocco casino owner in 1941 she...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/casablanca,$pent,8.0
4,Boyhood,NC-17,7.5,5,Drama,Richard Linklater,2014,IFC Films,Bonnie Cross,165,"Filmed over 12 years with the same cast, Richa...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/boyhood,'71,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14208,The Garbage Pail Kids Movie,R,0.7,14486,Action & Adventure,Rod Amateau,1987,Atlantic Releasing Corporation,Anthony Newly,100,Dodger must confront the struggles of life as ...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/the-garbage-p...,NaN,NaN
14209,United Passions,NR,0.7,14487,Drama,Frédéric Auburtin,2015,Screen Media Films,untagged,110,"Three men—Jules Rimet (Gérard Depardieu), Joao...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/united-passions,NaN,NaN
14210,Bio-Dome,R,7.5,14488,Comedy,Jason Bloom,1996,Metro-Goldwyn-Mayer (MGM),Pauly Shore,88,Five brave scientists are forced to face life ...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/bio-dome,NaN,NaN
14211,Chaos,NR,2.5,14489,Drama,David DeFalco,2005,Dinsdale Releasing,untagged,74,Quite definitely one of the most brutal displa...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/chaos,NaN,NaN


In [114]:
cols=pd.Series(df_metacritic_final.columns)
for dup in df_metacritic_final.columns[df_metacritic_final.columns.duplicated(keep=False)]: 
    cols[df_metacritic_final.columns.get_loc(dup)] = ([dup + '.' + str(d_idx) 
                                     if d_idx != 0 
                                     else dup 
                                     for d_idx in range(df_metacritic_final.columns.get_loc(dup).sum())]
                                    )
df_metacritic_final.columns=cols
df_metacritic_final

,title,age_rating,rating,rank,genre,director,year,producer,actor,runtime,description,img,url,title.1,ratecount
0,Citizen Kane,R,8.4,1,Drama,Orson Welles,1941,RKO Radio Pictures,Joseph Cotten,119,"Following the death of a publishing tycoon, ne...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/citizen-kane,!Women Art Revolution,11.0
1,The Godfather,NC-17,9.2,2,Drama,Francis Ford Coppola,1972,Paramount Pictures,Al Pacino,175,Francis Ford Coppola's epic features Marlon Br...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/the-godfather,#Horror,14.0
2,Rear Window,G,8.8,3,Mystery & Suspense,Alfred Hitchcock,1954,Paramount Pictures,Frank Cady,112,A wheelchair-bound photographer spies on his n...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/rear-window,$9.99,17.0
3,Casablanca,R,8.9,4,Drama,Michael Curtiz,1943,Warner Bros.,Humphrey Bogart,102,"A Casablanca, Morocco casino owner in 1941 she...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/casablanca,$pent,8.0
4,Boyhood,NC-17,7.5,5,Drama,Richard Linklater,2014,IFC Films,Bonnie Cross,165,"Filmed over 12 years with the same cast, Richa...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/boyhood,'71,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14208,The Garbage Pail Kids Movie,R,0.7,14486,Action & Adventure,Rod Amateau,1987,Atlantic Releasing Corporation,Anthony Newly,100,Dodger must confront the struggles of life as ...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/the-garbage-p...,NaN,NaN
14209,United Passions,NR,0.7,14487,Drama,Frédéric Auburtin,2015,Screen Media Films,untagged,110,"Three men—Jules Rimet (Gérard Depardieu), Joao...",https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/united-passions,NaN,NaN
14210,Bio-Dome,R,7.5,14488,Comedy,Jason Bloom,1996,Metro-Goldwyn-Mayer (MGM),Pauly Shore,88,Five brave scientists are forced to face life ...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/bio-dome,NaN,NaN
14211,Chaos,NR,2.5,14489,Drama,David DeFalco,2005,Dinsdale Releasing,untagged,74,Quite definitely one of the most brutal displa...,https://static.metacritic.com/images/products/...,https://www.metacritic.com/movie/chaos,NaN,NaN


In [115]:
df_metacritic_final.drop(columns=['title.1'], inplace=True)

# Loại bỏ các trường không cần thiết và tạo trường movie_id

In [124]:
df_metacritic_final2 = df_metacritic_final[['title', 'age_rating', 'rating', 'genre', 'director', 'year',
                                            'actor', 'runtime', 'description', 'img', 'url', 'ratecount']]

In [8]:
df_metacritic_final2["movie_id"]=["meta"+str(idx) for idx in range(len(df_metacritic_final2))]
title_id_dict = {}
for i in range(len(df_metacritic_final2)):
    title_id_dict[df_metacritic_final2.iloc[i]["title"]]=df_metacritic_final2.iloc[i]["movie_id"]

# Tạo thêm trường movie_id trong bảng review

In [9]:
def create_movie_id(title):
    # Có khả năng link đó không tồn tại trong bảng movie
    if title in title_id_dict.keys():
        return title_id_dict[title]
    else:
        return None
df_metacritic_review["movie_id"]=[create_movie_id(i) for i in df_metacritic_review["title"]]

In [10]:
print("Số đánh giá không rõ phim của bảng review trong metacritic: "+str(sum(df_metacritic_review["movie_id"].isna())))

Số đánh giá không rõ phim của bảng review trong metacritic: 0


# Lưu dữ liệu chuyển đổi

In [12]:
df_metacritic_final_movie = df_metacritic_final2[['movie_id', 'title', 'age_rating', 'genre', 'director', 'year',
                                                  'actor', 'runtime', 'description']]
df_metacritic_final_rating = df_metacritic_final2[['movie_id','rating', 'img', 'url', 'ratecount']]
df_metacritic_final_review = df_metacritic_review[['user', 'type', 'grade', 'review', 'spoilers', 'language', 'movie_id']]
df_metacritic_final_movie.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/metacritic/preprocess/movie.csv",index=False)
df_metacritic_final_rating.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/metacritic/preprocess/metacritic_rating.csv",index=False)
df_metacritic_final_review.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/metacritic/preprocess/metacritic_review.csv",index=False)